In [9]:
import os
import getpass

In [23]:
# from langchain_community.chat_models.ollama import ChatOllama
# model = ChatOllama(model="llama3")

In [3]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(model="llama3:70b", format="json", temperature=0)

In [4]:
print(model.invoke("Tell me a joke"))

content="Why don't scientists trust atoms? Because they make up everything!" id='run-3d4b2770-245d-4b79-a173-dbdb996d7298-0'


## Main

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain.agents import create_json_chat_agent, AgentExecutor

In [12]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [16]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717492321, 'localtime': '2024-06-04 2:12'}, 'current': {'last_updated_epoch': 1717491600, 'last_updated': '2024-06-04 02:00', 'temp_c': 9.0, 'temp_f': 48.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1015.0, 'pressure_in': 29.96, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 0, 'feelslike_c': 8.7, 'feelslike_f': 47.7, 'windchill_c': 12.4, 'windchill_f': 54.3, 'heatindex_c': 12.4, 'heatindex_f': 54.3, 'dewpoint_c': 9.1, 'dewpoint_f': 48.4, 'vis_km': 9.7, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 5.5, 'gust_kph': 8.9}}"}, {'url': 'https://world-weather.info/forecast/usa/san_francisco/

In [1]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat-json")

print(prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat-json', 'lc_hub_commit_hash': '9c1258e8aa8ce33bebbd62e077c143d0b06c81f3c7de732187ee61c70c1254c7'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by Open

In [57]:
# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
# agent_executor= create_react_agent(model, tools, checkpointer=memory)
agent= create_json_chat_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)


In [52]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"input" : "hi im bob! and i live in sf"}, config
):
    print(chunk)
    print("----")

{'actions': [AgentAction(tool='tavily_search_results_json', tool_input='San Francisco news', log='{ \n  "action": "tavily_search_results_json", \n  "action_input": "San Francisco news" \n}')], 'messages': [AIMessage(content='{ \n  "action": "tavily_search_results_json", \n  "action_input": "San Francisco news" \n}')]}
----
{'steps': [AgentStep(action=AgentAction(tool='tavily_search_results_json', tool_input='San Francisco news', log='{ \n  "action": "tavily_search_results_json", \n  "action_input": "San Francisco news" \n}'), observation=[{'url': 'https://abc7news.com/san-francisco/', 'content': "Get the latest news and updates on San Francisco, covering topics such as education, crime, business, culture and more. Watch videos, read stories and browse photos from ABC7, the Bay Area's leading news station."}, {'url': 'https://www.sfchronicle.com/', 'content': "Get the latest news and updates on San Francisco and the Bay Area, including crime, politics, real estate, arts and entertainmen

### Part 2

In [58]:
model = model.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [59]:
model.invoke("what is the weather in Boston?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Boston, MA", "unit": "fahrenheit"}'}}, id='run-2113385f-b42b-4c6c-a60a-25a4827f42f5-0')

In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


# Schema for structured response
class Person(BaseModel):
    name: str = Field(description="The person's name", required=True)
    height: float = Field(description="The person's height", required=True)
    hair_color: str = Field(description="The person's hair color")


# Prompt template
prompt = PromptTemplate.from_template(
    """Alex is 5 feet tall. 
Claudia is 1 feet taller than Alex and jumps higher than him. 
Claudia is a brunette and Alex is blonde.

Human: {question}
AI: """
)

# Chain
structured_llm = model.with_structured_output(Person)
chain = prompt | structured_llm

In [6]:
alex = chain.invoke("Describe Alex")
alex

Person(name='Alex', height=5.0, hair_color='blonde')

In [7]:
claudia = chain.invoke("Describe Claudia")
claudia

Person(name='Claudia', height=6.0, hair_color='brown')

# Load Local Models using LLamacpp
### Didn't like the response

In [1]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [2]:
model_path = '/Users/vinaytanwer/Desktop/Projects/Chatbots/LLM_checkpoints/Quantfactory/Meta-Llama-3-8B.Q4_0.gguf'

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


n_gpu_layers = -1  # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
# # Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.5,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,  # Verbose is required to pass to the callback manager
    streaming=False
)

In [3]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a helpful assistant. Answer the user
    question honestly. Format of the answer should be in json with 1 key as 'answer'.
    User will provide a statement, reply to the user honestly.
    Do not include any pre-amble before the assistant response.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the user statement: {statement} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["statement"],
)

test_llm = prompt | llm
question = "Tell me a joke about superman"
print(test_llm.invoke({"statement": question}))

 {
        "answer": "Why did superman never win an oscar? Because he was always too super for his own good! "
     }



In [4]:
from langchain_core.tools import tool

@tool
def magic_function(x:float, y:float) -> float:
    "Given two float numbers x and y in arguments, it returns the multiplication of x and y"
    return (x*y)

magic_function.invoke({"x":2.3, "y":3})

6.8999999999999995

## Ollama functions Testing

In [45]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="llama3", temperature=0) #, format="json")

In [32]:
import os
dir_path = "/Users/vinaytanwer/Desktop/Projects/Chatbots/langchain/apps/mj-app/packages/neo4j-advanced-rag"
mj_docs_dir = os.path.join(dir_path, "mj_scraper/mj_docs/final")
file_name = "terms-of-service.txt"
txt_path = os.path.join(mj_docs_dir, file_name)

In [33]:
from langchain_text_splitters import TokenTextSplitter
from langchain_community.document_loaders import TextLoader

# Load the text file
loader = TextLoader(str(txt_path))
documents = loader.load()

# Ingest Parent-Child node pairs
parent_splitter = TokenTextSplitter(chunk_size=1500, chunk_overlap=250)
parent_documents = parent_splitter.split_documents(documents)


In [34]:
len(parent_documents)

3

In [8]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on the information from the text"
        ),
    )

questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating hypothetical questions based on the information "
                "found in the text. Make sure to provide full context in the generated "
                "questions."
            ),
        ),
        (
            "human",
            (
                "Use the given format to generate hypothetical questions from the "
                "following input: {input}"
            ),
        ),
    ]
)

question_chain = questions_prompt | llm.with_structured_output(Questions)


In [15]:
text = parent_documents[2].page_content
# print(text)

In [16]:

questions = question_chain.invoke({"input":text})
print(questions.questions)


['What is the time frame for resolving a dispute within this agreement?', 'How does the arbitration process work in this agreement?', 'What are the consequences if a party fails to resolve a dispute within 30 days?', 'What is the governing law of this agreement?', 'What happens if a party violates the Community Guidelines or other inappropriate use of the Service?', 'Can a party cancel their plan at any time, and what are the implications?', "How does the indemnification process work in case of third-party claims arising from a party's use of the Services and Assets?"]


In [46]:

# Define Summary Generation Retrieval Chain

class Summary(BaseModel):
    """Generating hypothetical questions about text."""

    summary: str = Field(
        description=(
            "Generated summary for the information from the text"
        ),
    )

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are expert in Summary writing. You are generating accurate summary for the input provided in the text."
                "Output should be a single key 'summary'"
            ),
        ),
        (
            "human",
            (
                "Generate summary for the following input: {input}"
            ),
        ),
    ]
)

summary_chain = summary_prompt | llm.with_structured_output(Summary)
# summary_chain = summary_prompt | llm

In [47]:
text = parent_documents[1].page_content
summary = summary_chain.invoke({"input":text})
summary

Summary(summary="By using Midjourney's Services, users grant a perpetual license to reproduce, prepare derivative works, publicly display, publicly perform, sublicense, and distribute text and image prompts. The license survives termination of the Agreement by any party. Users also grant permission for others to remix their images and prompts in public settings. Stealth mode is available for certain subscription plans, but does not guarantee complete privacy. The Services have a DMCA and Takedowns Policy, which outlines procedures for reporting copyright infringement. Disputes will be resolved through binding arbitration.")